In [1]:
import pandas as pd
import numpy as np

In [2]:
#set variables
MA_length = 20
exit_signal = 0

#read in crude oil futures data and daily risk-free rates data (obtained from WRDS)
crude = pd.read_csv("Crude_Oil.txt",sep="\t")
crude.Date = pd.to_datetime(crude.Date,dayfirst=True)
Rf = pd.read_csv("Rf.csv",header=0,names=['Date','Rf'])
Rf.Date = pd.to_datetime(Rf.Date)

In [3]:
#get relevant dates for risk free rate
Rf = Rf[(Rf.Date > '1989-08-10') & (Rf.Date < '2014-02-18')]
Rf = Rf.reset_index(drop=True)
Rf

,Date,Rf
0,1989-08-11,1.920000e-04
1,1989-08-14,1.930000e-04
2,1989-08-15,1.830000e-04
3,1989-08-16,1.810000e-04
4,1989-08-17,1.730000e-04
...,...,...
6121,2014-02-10,1.530000e-06
6122,2014-02-11,1.530000e-06
6123,2014-02-12,6.940000e-07
6124,2014-02-13,-1.390000e-07


In [4]:
crude

,Date,RCLC1,RCLC2,RCLC3,RCLC4,RCLC5,RCLC6,RCLC12,LLCC1,LLCC2,LLCC3,LLCC4,LLCC5,LLCC6,LLCC12
0,1989-08-10,18.59,18.29,18.17,18.10,18.01,17.95,17.73,16.97,16.88,16.84,16.50,16.53,16.53,NaN
1,1989-08-11,18.48,18.18,18.05,17.96,17.87,17.81,17.60,16.91,16.81,16.70,16.63,16.63,16.63,NaN
2,1989-08-14,18.58,18.22,18.02,17.90,17.79,17.70,17.48,16.79,16.67,16.55,16.55,16.55,16.55,NaN
3,1989-08-15,18.68,18.29,18.07,17.96,17.84,17.75,17.50,16.87,16.78,16.59,16.55,16.55,16.55,NaN
4,1989-08-16,18.97,18.49,18.25,18.13,18.02,17.92,17.67,16.96,16.83,16.66,16.55,16.60,16.60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6394,2014-02-12,100.37,99.92,99.17,98.37,97.59,96.74,91.97,108.79,108.35,108.03,107.68,107.29,106.87,104.20
6395,2014-02-13,100.35,100.05,99.31,98.54,97.75,96.88,92.05,108.73,108.52,108.19,107.82,107.42,107.00,104.31
6396,2014-02-14,100.30,100.13,99.47,98.73,97.95,97.08,92.19,109.08,108.76,108.39,108.00,107.55,106.98,104.27
6397,2014-02-17,100.30,100.13,99.47,98.73,97.95,97.08,92.19,109.18,108.88,108.54,108.16,107.71,107.15,104.48


In [5]:
#drop columns we won't be using
crude = crude.drop(columns = ["RCLC2","RCLC3","RCLC4","RCLC5","RCLC6","RCLC12","LLCC2","LLCC3","LLCC4","LLCC5","LLCC6","LLCC12"])
crude

,Date,RCLC1,LLCC1
0,1989-08-10,18.59,16.97
1,1989-08-11,18.48,16.91
2,1989-08-14,18.58,16.79
3,1989-08-15,18.68,16.87
4,1989-08-16,18.97,16.96
...,...,...,...
6394,2014-02-12,100.37,108.79
6395,2014-02-13,100.35,108.73
6396,2014-02-14,100.30,109.08
6397,2014-02-17,100.30,109.18


In [6]:
#create the column for the spread
crude['Spread'] = crude.RCLC1 - crude.LLCC1
crude

,Date,RCLC1,LLCC1,Spread
0,1989-08-10,18.59,16.97,1.62
1,1989-08-11,18.48,16.91,1.57
2,1989-08-14,18.58,16.79,1.79
3,1989-08-15,18.68,16.87,1.81
4,1989-08-16,18.97,16.96,2.01
...,...,...,...,...
6394,2014-02-12,100.37,108.79,-8.42
6395,2014-02-13,100.35,108.73,-8.38
6396,2014-02-14,100.30,109.08,-8.78
6397,2014-02-17,100.30,109.18,-8.88


In [7]:
#create columns for moving average, rolling standard deviation, and z-score for the current price level of the spread
crude['MA'+str(MA_length)] = crude.Spread.rolling(MA_length).mean()
crude['SD'+str(MA_length)] = crude.Spread.rolling(MA_length).std()
crude['Z'+str(MA_length)] = (crude.Spread-crude['MA'+str(MA_length)])/crude['SD'+str(MA_length)]
crude.head(25)

,Date,RCLC1,LLCC1,Spread,MA20,SD20,Z20
0,1989-08-10,18.59,16.97,1.62,NaN,NaN,NaN
1,1989-08-11,18.48,16.91,1.57,NaN,NaN,NaN
2,1989-08-14,18.58,16.79,1.79,NaN,NaN,NaN
3,1989-08-15,18.68,16.87,1.81,NaN,NaN,NaN
4,1989-08-16,18.97,16.96,2.01,NaN,NaN,NaN
5,1989-08-17,18.68,16.89,1.79,NaN,NaN,NaN
6,1989-08-18,18.80,16.85,1.95,NaN,NaN,NaN
7,1989-08-21,19.08,16.83,2.25,NaN,NaN,NaN
8,1989-08-22,19.08,17.01,2.07,NaN,NaN,NaN
9,1989-08-23,18.80,16.95,1.85,NaN,NaN,NaN


In [8]:
# create signal column which determines when we are long/short the spread

# determine initial entry points, which is when z-score is above 2 or below -2
crude["Signal"] = [1 if x < -2 else (-1 if x > 2 else 0) for x in crude.Z20]

# fill in signal columns until the point where we receive an exit signal (determined by "exit_signal" variable)
for i in range(1,len(crude.index)):
    if crude.iloc[i-1,7] == -1:
        if crude.iloc[i,6] > -1*exit_signal:
            crude.iloc[i,7] = -1
    elif crude.iloc[i-1,7] == 1:
        if crude.iloc[i,6] < exit_signal:
            crude.iloc[i,7] = 1
            
crude.head(50)


,Date,RCLC1,LLCC1,Spread,MA20,SD20,Z20,Signal
0,1989-08-10,18.59,16.97,1.62,NaN,NaN,NaN,0
1,1989-08-11,18.48,16.91,1.57,NaN,NaN,NaN,0
2,1989-08-14,18.58,16.79,1.79,NaN,NaN,NaN,0
3,1989-08-15,18.68,16.87,1.81,NaN,NaN,NaN,0
4,1989-08-16,18.97,16.96,2.01,NaN,NaN,NaN,0
5,1989-08-17,18.68,16.89,1.79,NaN,NaN,NaN,0
6,1989-08-18,18.80,16.85,1.95,NaN,NaN,NaN,0
7,1989-08-21,19.08,16.83,2.25,NaN,NaN,NaN,0
8,1989-08-22,19.08,17.01,2.07,NaN,NaN,NaN,0
9,1989-08-23,18.80,16.95,1.85,NaN,NaN,NaN,0


In [9]:
#calculate returns; we will use the average price level of the two contracts as our investment/"employed capital"
crude['Ret'] = crude.Spread.diff().shift(-1)/(crude.RCLC1+crude.LLCC1)/2
crude.Ret = crude.Ret*crude.Signal

In [10]:
crude.head(50)

,Date,RCLC1,LLCC1,Spread,MA20,SD20,Z20,Signal,Ret
0,1989-08-10,18.59,16.97,1.62,NaN,NaN,NaN,0,-0.000000
1,1989-08-11,18.48,16.91,1.57,NaN,NaN,NaN,0,0.000000
2,1989-08-14,18.58,16.79,1.79,NaN,NaN,NaN,0,0.000000
3,1989-08-15,18.68,16.87,1.81,NaN,NaN,NaN,0,0.000000
4,1989-08-16,18.97,16.96,2.01,NaN,NaN,NaN,0,-0.000000
5,1989-08-17,18.68,16.89,1.79,NaN,NaN,NaN,0,0.000000
6,1989-08-18,18.80,16.85,1.95,NaN,NaN,NaN,0,0.000000
7,1989-08-21,19.08,16.83,2.25,NaN,NaN,NaN,0,-0.000000
8,1989-08-22,19.08,17.01,2.07,NaN,NaN,NaN,0,-0.000000
9,1989-08-23,18.80,16.95,1.85,NaN,NaN,NaN,0,-0.000000


In [11]:
#get summary statistics for returns
returns = crude.Ret[crude.Ret !=0]
returns = returns.dropna()
stats = returns.describe()
stats

count    2101.000000
mean        0.000821
std         0.004419
min        -0.025578
25%        -0.001047
50%         0.000243
75%         0.001738
max         0.055156
Name: Ret, dtype: float64

In [12]:
#get average daily risk free rate over the period
stats2 = Rf.Rf.describe()
stats2

count    6.126000e+03
mean     8.460137e-05
std      6.088589e-05
min     -2.080000e-07
25%      2.540000e-05
50%      9.230000e-05
75%      1.340000e-04
max      2.240000e-04
Name: Rf, dtype: float64

In [13]:
sharpe = (stats[1]-stats2[1])/stats[2]*np.sqrt(252)
sharpe

2.646751423499015